In [5]:
import pandas as pd
from sklearn.cluster import AffinityPropagation

In [80]:
from mb_functions import *

In [66]:
SPOTIPY_CLIENT_ID= 'a5d9928d7d874cebafc8a8a632277854'
SPOTIPY_CLIENT_SECRET= 'a3341b047daa4007a5bc3ad2fc9413c7'
SPOTIPY_REDIRECT_URI = 'http://127.0.0.1:9090'
SCOPE = 'user-top-read user-library-read playlist-modify-public'

In [67]:
auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_REDIRECT_URI, scope=SCOPE)
token = auth_manager.get_access_token()['access_token']

/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_63715/1028136909.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = auth_manager.get_access_token()['access_token']


In [3]:
df = pd.read_csv('assets/df_pca.csv')

In [12]:
cols = [ 'Danceability', 'Energy',
            'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
           'Valence', 'Tempo'] + ['PC'+str(i+1) for i in range(5)]

In [14]:
groovy = df[df.search_id == 'groovy']
#Model top songs 
model = AffinityPropagation(damping = .5)
model_fit = model.fit(groovy[cols])
#give clusters
# top_vibe_songs['clustered'] = model_fit.predict(top_vibe_songs[cols])
groovy['clustered'] = model_fit.predict(groovy[cols])

In [15]:
#Model top songs 
model = AffinityPropagation(damping = .5)
model_fit = model.fit(groovy[cols])

In [23]:
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn.impute import SimpleImputer

# Assuming df is your DataFrame and cols is a list of column names to use for clustering

# Initialize an imputer (this example uses the median value of each column)
imputer = SimpleImputer(strategy='median')

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through each unique search_id
for search_id in df['search_id'].unique():
    # Filter the DataFrame for the current search_id
    segment = df[df.search_id == search_id]
    
    # Perform imputation on the selected columns
    segment[cols] = imputer.fit_transform(segment[cols])
    
    # Perform clustering
    model = AffinityPropagation(damping=0.5)
    model_fit = model.fit(segment[cols])
    
    # Assign the cluster labels to the 'clustered' column in the groovy DataFrame
    segment['clustered'] = model_fit.predict(segment[cols])
    
    # Append the processed DataFrame to the result_df
    result_df = pd.concat([result_df, segment], ignore_index=True)

# result_df now contains the original data along with a 'clustered' column for cluster labels


/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_63715/1380934054.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segment[cols] = imputer.fit_transform(segment[cols])
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_63715/1380934054.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segment['clustered'] = model_fit.predict(segment[cols])
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_63715/1380934054.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [39]:
import pandas as pd
import requests

# Function to fetch recommendations from Spotify
def get_spotify_recommendations(access_token, seed_tracks, min_popularity, max_popularity):
    url = "https://api.spotify.com/v1/recommendations"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "seed_tracks": ','.join(seed_tracks),
        "limit": 100  # Fetch more to manually filter by popularity
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        recs_data = []
        for track in response.json()['tracks']:
            popularity = track['popularity']
            if min_popularity <= popularity <= max_popularity:
                recs_data.append({
                    'track_id': track['id'],
                    'track_name': track['name'],
                    'artist_id': track['artists'][0]['id'],  # Assuming taking the first artist
                    'popularity_range': f"{min_popularity}-{max_popularity}"  # Track the popularity range
                })
                if len(recs_data) >= 3:  # Limiting to 3 recommendations per popularity range
                    break
        return pd.DataFrame(recs_data)
    else:
        print(f"Error fetching recommendations: {response.status_code}")
        return pd.DataFrame()

In [82]:
result_df.head()

,search_id,playlist_number,playlist_id,playlist_name,track_id,track_name,song_count,Danceability,Energy,Loudness,...,Artist ID,Artist Followers,Artist Genres,Artist Popularity,PC1,PC2,PC3,PC4,PC5,clustered
0,groovy,1,37i9dQZF1EIhMHNZW8S7ky,Groovy Mix,3qiX7QXv6w6y4wsr5KP6U1,Sea Sick,2.0,0.817,0.577,-8.630,...,2jbd7OqeJJd1hz81vOXwwW,100054.0,['modern indie pop'],47.0,-0.145481,0.190767,-0.228742,0.047288,-0.047504,4
1,groovy,1,37i9dQZF1EIhMHNZW8S7ky,Groovy Mix,0YMe6PHRbeDcN7KJdCG0bW,The Dress,2.0,0.747,0.531,-5.990,...,0knGpCTbmG4ctl1wzYRZs4,325795.0,['bedroom soul'],56.0,-0.144511,0.188991,-0.225415,0.046752,-0.046976,2
2,groovy,1,37i9dQZF1EIhMHNZW8S7ky,Groovy Mix,5D7AFGloFpXtdT2PiKTEC3,Alien Love Call,2.0,0.606,0.583,-8.389,...,2qnpHrOzdmOo1S4ox3j17x,506242.0,"['hardcore punk', 'modern melodic hardcore']",59.0,-0.143668,0.187491,-0.222184,0.044699,-0.044399,0
3,groovy,1,37i9dQZF1EIhMHNZW8S7ky,Groovy Mix,6nXiaXqSZAJuOG6q66mZV1,Lil Baby Crush,3.0,0.858,0.435,-9.611,...,3DGlTwdM5Dim9XQipb3jMf,154223.0,"['indie hip hop', 'indie r&b']",60.0,-0.145489,0.191042,-0.229142,0.049109,-0.047403,1
4,groovy,1,37i9dQZF1EIhMHNZW8S7ky,Groovy Mix,7to68V64Cu6zk0UDo5tyw3,Confidence,3.0,0.590,0.562,-9.607,...,18lpwfiys4GtdHWNUu9qQr,545291.0,"['australian reggae fusion', 'australian surf ...",66.0,-0.143689,0.187475,-0.222595,0.045873,-0.046036,1


### Current Spot - Need to Implement Below

In [ ]:
# Artists Associate from top 30 to find what type of listener

In [ ]:
# Song Meaning - Genius - Open AI Interpretation or another free model
# Tempo - Spotify BPM
# Atomosphere & Tone - Spotify
# Genres - PCAs
# Music Purpose - Open AI from playlist name
# Emotion Pairing - Open AI from Playlist name

In [ ]:
#Imputate Missing Data

#### Add Recs Below (Ignore)

- Going to add more songs by pulling recs for each search_id & cluster(sub vibe) 

In [40]:
recommendations_df = pd.DataFrame()

In [47]:
import time

In [68]:
# Your access token
access_token = token

popularity_ranges = [(20, 40), (40, 70), (70, 100)]

# Loop through each unique search_id and cluster combination
for (search_id, cluster) in result_df.iloc[786:][['search_id', 'clustered']].drop_duplicates().itertuples(index=False):
    time.sleep(1)
    # Filter the DataFrame for the current search_id and cluster
    current_group = result_df[(result_df['search_id'] == search_id) & (result_df['clustered'] == cluster)]
    
    # Sample 5 track_ids from the current group, or fewer if the group is smaller
    sample_size = min(5, len(current_group))
    sampled_tracks = current_group['track_id'].sample(n=sample_size, random_state=1).tolist()
    
    # For each sampled track_id, get recommendations for each popularity range
    for min_pop, max_pop in popularity_ranges:
        recs = get_spotify_recommendations(access_token, sampled_tracks, min_pop, max_pop)
        if not recs.empty:
            recs['search_id'] = search_id
            recs['clustered'] = cluster
            recommendations_df = pd.concat([recommendations_df, recs], ignore_index=True)

# Now you would merge recommendations_df back into your original DataFrame as needed

# Merge the recommendations back into the original DataFrame
# Here, you might need a more sophisticated merging strategy depending on your exact requirements and DataFrame structure
# This example simply appends the new recommendations assuming they do not overlap with existing rows
# final_df = pd.concat([df, recommendations_df], ignore_index=True, sort=False).fillna(value=np.nan)

# Ensure the final DataFrame has the structure you expect, especially after the merge


Error fetching recommendations: 429
Error fetching recommendations: 429
Error fetching recommendations: 429
Error fetching recommendations: 429


KeyboardInterrupt: 

In [55]:
result_df.iloc[786:]

,search_id,playlist_number,playlist_id,playlist_name,track_id,track_name,song_count,Danceability,Energy,Loudness,...,Artist ID,Artist Followers,Artist Genres,Artist Popularity,PC1,PC2,PC3,PC4,PC5,clustered
786,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,6.0,0.843,0.806,-5.946,...,1G9G7WwrXka3Z1r7aIDjI7,2497325.0,"['atl hip hop', 'dirty south rap', 'hip hop', ...",73.0,1.276198,-0.064632,0.093209,0.205965,0.293680,10
787,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,6etwirSInfuJrSOlogS6Sh,American Boy,3.0,0.693,0.729,-2.990,...,5T0MSzX9RC5NA6gAI6irSn,276267.0,"['neo soul', 'r&b']",65.0,-0.123324,0.110730,-0.209107,0.259508,0.111916,1
788,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,1zi7xx7UVEFkmKfv06H8x0,One Dance,2.0,0.792,0.625,-5.609,...,3TVXtAsR1Inumwj472S9r4,84097254.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",96.0,1.514303,-0.214995,0.261128,-0.545489,-0.535601,8
789,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,6WpZwkzyjINc6wDghg8Gzv,Girls,2.0,0.922,0.831,-1.918,...,2mqiqsaX4LzFnUP7PmHGAb,18444.0,[''],35.0,-0.169052,0.237744,-0.328901,0.098007,-0.107173,2
790,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,6qMXubogFdMrvVMJwqATzd,Jordan Belfort,2.0,0.860,0.719,-4.325,...,0HjYETXAvcL6mzaKjAmH2K,18642.0,[''],45.0,-0.169052,0.237744,-0.328901,0.098007,-0.107173,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25143,don't give a fuck,13,37i9dQZF1E8KEQvIztpWGc,Just Don't Give A Fuck Radio,2q8lHMMso6wx6pb5uwqzOs,Two Dope Boyz (In a Cadillac),2.0,0.809,0.718,-4.398,...,1G9G7WwrXka3Z1r7aIDjI7,2497325.0,"['atl hip hop', 'dirty south rap', 'hip hop', ...",73.0,1.276198,-0.064632,0.093209,0.205965,0.293680,4
25144,don't give a fuck,13,37i9dQZF1E8KEQvIztpWGc,Just Don't Give A Fuck Radio,0N3W5peJUQtI4eyR6GJT5O,King Kunta,2.0,0.884,0.657,-4.569,...,2YZyLoL8N0Wb9xBt1NhZWg,26793764.0,"['conscious hip hop', 'hip hop', 'rap', 'west ...",88.0,1.087644,0.006625,0.026991,-0.420767,-0.314608,4
25145,don't give a fuck,13,37i9dQZF1E8KEQvIztpWGc,Just Don't Give A Fuck Radio,2ImuQo1g14CTR9hZAZD3aQ,This D.J.,2.0,0.804,0.443,-10.268,...,2B4ZHz4QDWJTXPFPgO5peE,2338774.0,"['g funk', 'gangster rap', 'hardcore hip hop',...",62.0,0.471302,0.121602,-0.095557,-0.279656,-0.292066,4
25146,don't give a fuck,13,37i9dQZF1E8KEQvIztpWGc,Just Don't Give A Fuck Radio,0EtirusGt3LivUmALXSMVe,Role Model,2.0,0.865,0.589,-4.224,...,7dGJo4pcD2V6oG8kP0tJRR,80833874.0,"['detroit hip hop', 'hip hop', 'rap']",90.0,1.018103,0.004524,0.023234,-0.311795,-0.226979,4


In [54]:
result_df[(result_df.search_id == 'party')&(result_df.clustered == 10)]

,search_id,playlist_number,playlist_id,playlist_name,track_id,track_name,song_count,Danceability,Energy,Loudness,...,Artist ID,Artist Followers,Artist Genres,Artist Popularity,PC1,PC2,PC3,PC4,PC5,clustered
786,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,6.0,0.843,0.806,-5.946,...,1G9G7WwrXka3Z1r7aIDjI7,2497325.0,"['atl hip hop', 'dirty south rap', 'hip hop', ...",73.0,1.276198,-0.064632,0.093209,0.205965,0.293680,10
792,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,3CA9pLiwRIGtUBiMjbZmRw,Nice For What,2.0,0.585,0.909,-6.474,...,3TVXtAsR1Inumwj472S9r4,84097254.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",96.0,1.514303,-0.214995,0.261128,-0.545489,-0.535601,10
793,party,1,37i9dQZF1EIdzRg9sDFEY3,Party Mix,7iQXYTyuG13aoeHxGG28Nh,PERRO NEGRO,2.0,0.911,0.778,-2.248,...,4q3ewBCX7sLwd24euuV69X,78304191.0,"['reggaeton', 'trap latino', 'urbano latino']",94.0,-0.180807,0.278172,-0.447615,0.328893,-0.291084,10
813,party,3,29dk0RaLOxL9tDsRnrrvDM,Partyyy Duffyyy,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,2.0,0.911,0.712,-5.105,...,4q3ewBCX7sLwd24euuV69X,78304191.0,"['reggaeton', 'trap latino', 'urbano latino']",94.0,-0.180807,0.278172,-0.447615,0.328893,-0.291084,10
834,party,4,5W7CzWheiXkRGrQfBX6UqM,help i’m on aux at a party,5FkoSXiJPKTNyYgALRJFhD,ORANGE SODA,2.0,0.830,0.564,-7.822,...,5SXuuuRpukkTvsLuUknva1,2112982.0,"['hip hop', 'rap']",78.0,0.998623,0.006098,0.020151,-0.314561,-0.232814,10
838,party,4,5W7CzWheiXkRGrQfBX6UqM,help i’m on aux at a party,1PS1QMdUqOal0ai3Gt7sDQ,Gold Digger,3.0,0.629,0.696,-5.572,...,5K4W6rqBFWDnAN6FQUkS6x,23798037.0,"['chicago rap', 'hip hop', 'rap']",94.0,1.063316,0.007392,0.018514,-0.345915,-0.249826,10
854,party,6,5xS3Gi0fA3Uo6RScucyct6,Party Songs 🥳 2010-2024 Hits 🎉,2a1o6ZejUi8U3wzzOtCOYw,Danza Kuduro,3.0,0.400,0.915,-4.890,...,33ScadVnbm2X8kkUqOkC6Z,10762269.0,"['latin hip hop', 'puerto rican pop', 'reggaet...",81.0,-0.186348,0.290771,-0.478505,0.368502,-0.321747,10
894,party,6,5xS3Gi0fA3Uo6RScucyct6,Party Songs 🥳 2010-2024 Hits 🎉,3AYcyxEACnmE6d96RPubID,Thrift Shop (feat. Wanz),2.0,0.781,0.526,-6.985,...,5BcAKTbp20cv7tC5VqPFoC,2507048.0,"['pop rap', 'seattle hip hop']",73.0,0.161789,0.058716,-0.083693,0.165678,0.013614,10
898,party,6,5xS3Gi0fA3Uo6RScucyct6,Party Songs 🥳 2010-2024 Hits 🎉,5vlEg2fT4cFWAqU5QptIpQ,Replay,3.0,0.706,0.751,-6.323,...,5tKXB9uuebKE34yowVaU3C,911286.0,"['dance pop', 'pop rap', 'post-teen pop']",64.0,0.043978,-0.390801,0.126457,0.444652,0.094747,10
899,party,6,5xS3Gi0fA3Uo6RScucyct6,Party Songs 🥳 2010-2024 Hits 🎉,5HQVUIKwCEXpe7JIHyY734,"Young, Wild & Free (feat. Bruno Mars)",3.0,0.715,0.655,-6.425,...,7hJcb9fa4alzcOq3EaNPoG,10984486.0,"['g funk', 'gangster rap', 'hip hop', 'pop rap...",81.0,1.410274,-0.134002,0.179123,-0.239860,-0.204028,10


In [48]:
recommendations_df.tail()

,track_id,track_name,artist_id,popularity_range,search_id,clustered
540,79lmvU7Qmc8QpXilbSk37z,Game Over,4Q5sPmM8j4SpMqL4UA1DtS,40-70,party,10
541,1r3myKmjWoOqRip99CmSj1,Don't Wanna Go Home,07YZf4WDAMNwqr4jfgOZ8y,40-70,party,10
542,5AyL2kgLtTWEu3qO3B9SqK,Like A G6,698hF4vcwHwPy8ltmXermq,70-100,party,10
543,53v2u51nq3CxiKUNphRzP1,Que Malo,4q3ewBCX7sLwd24euuV69X,70-100,party,10
544,2beS6dD5rovBxSA1zYoSHR,No Me Conoce - Remix,6nVcHLIgY5pE2YCl8ubca1,70-100,party,10


In [ ]:
final_df = pd.concat([df, recommendations_df], ignore_index=True, sort=False).fillna(value=np.nan)